# Sparseloop Tutorial


First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ./prelude.py --style=tree --animation=movie

# Convolution


## Configure convolution tensors

In [ ]:
W = 12
S = 3


tm3 = TensorMaker("sparseloop-convolution-1", autoload=True)

tm3.addTensor("F", rank_ids=["S"], shape=[S], density=0.5, color="green")
tm3.addTensor("I", rank_ids=["W"], shape=[W], density=1.0, color="blue")

tm3.displayControls()


## Create and display convolution tensors

In [ ]:
F_S = tm3.makeTensor("F")
I_W = tm3.makeTensor("I")

displayTensor(F_S)
displayTensor(I_W)


# Simple 1-D Convolution

$$ O_{q} = I_{(q+s)} \times F_{s} $$

In [ ]:
#
# Create a convolution tensors
#
S = getShape(tm3, "S")
W = getShape(tm3, "W")
Q = W-S+1

F_S = tm3.makeTensor("F")
I_W = tm3.makeTensor("I")

O_Q = Tensor(name="O", rank_ids=["Q"], shape=[Q])
uncompressTensor(O_Q)

#
# Display Tensors
#
print(f"S: {S}")
print(f"W: {W}")
print(f"Q: {Q}")

displayTensor(F_S)
displayTensor(I_W)
displayTensor(O_Q)

#
# Get root of tensors
#
i_w = I_W.getRoot()
f_s = F_S.getRoot()
o_q = O_Q.getRoot()

#
# Animation bookkeeeping
#
canvas = createCanvas(F_S, I_W, O_Q)

cycle = 0

#
# Traverse non-empty elements `input`
#
for q in range(Q):
    for s, f_val in f_s:
        w = q + s
        i_val = i_w.getPayload(w)
        
        o_q[q] += i_val * f_val
        
        #
        # Animation bookkeeping
        #
        canvas.addActivity((s,), (w,), (q,), spacetime=(0,cycle))
        cycle += 1
            
displayTensor(O_Q)
displayCanvas(canvas)

# Convolution - 1D - with output channels

$$ O_{m,q} = I_{(q+s)} \times F_{m,s} $$

## Configure convolution tensors

In [ ]:
M = 4
W = 12
S = 4

tm4 = TensorMaker("sparseloop-convolution-2", autoload=True)

tm4.addTensor("F", rank_ids=["M", "S"], shape=[M, S], density=0.5, color="green")
tm4.addTensor("I", rank_ids=["W"], shape=[W], density=1.0, color="blue")

tm4.displayControls()


## Create and display convolution tensors

In [ ]:
F_MS = tm4.makeTensor("F")
I_W = tm4.makeTensor("I")

displayTensor(F_MS)
displayTensor(I_W)


## Convolution

In [ ]:
#
# Create a convolution tensors
#

M = getShape(tm4, "M")
S = getShape(tm4, "S")
W = getShape(tm4, "W")
Q = W - S + 1

F_MS = tm4.makeTensor("F")
I_W = tm4.makeTensor("I")

O_MQ = Tensor(name="O", rank_ids=["M", "Q"], shape=[M, Q])
uncompressTensor(O_MQ)

#
# Display Tensors
#
print(f"M: {M}")
print(f"S: {S}")
print(f"W: {W}")
print(f"Q: {Q}")

displayTensor(F_MS)
displayTensor(I_W)
displayTensor(O_MQ)

#
# Get root of tensors
#
i_w = I_W.getRoot()
f_m = F_MS.getRoot()
o_m = O_MQ.getRoot()

#
# Animation bookkeeeping
#
canvas = createCanvas(F_MS, I_W, O_MQ)

cycle = 0

#
# Traverse non-empty elements `input`
#
for m, f_s in f_m:    
    for q in range(Q):
        for s, f_val in f_s:
        #
        # Traverse all output locations
        #
            w = q + s
            i_val = i_w.getPayload(w)

            o_m[m][q] += i_val * f_val
            #
            # Animation bookkeeping
            #
            canvas.addActivity((m,s), (w,), (m,q), spacetime=(0,cycle))
            cycle += 1
            
displayTensor(O_MQ)
displayCanvas(canvas)

## Convolution - spatial output channels

In [ ]:
#
# Create a convolution tensors
#

split = 2

F_MS = tm4.makeTensor("F")
F_M1M0S = F_MS.splitUniform(split)
I_W = tm4.makeTensor("I")

W = I_W.getShape("W")
S = F_MS.getShape("S")
Q = W-S+1

M = F_MS.getShape("M")
M0 = split
M1 = (M+split-1)//split

O_MQ = Tensor(name="O", rank_ids=["M", "Q"], shape=[M, Q])
uncompressTensor(O_MQ)
O_M1M0Q = O_MQ.splitUniform(split)

#
# Display tensors
#

print(f"M1: {M1}")
print(f"M0: {M0}")
print(f"W:  {W}")
print(f"Q:  {Q}")
print(f"S:  {S}")

displayTensor(F_M1M0S)
displayTensor(I_W)
displayTensor(O_M1M0Q)

#
# Get root of tensors
#
i_w = I_W.getRoot()
f_m1 = F_M1M0S.getRoot()
o_m1 = O_M1M0Q.getRoot()

#
# Animation bookkeeeping
#
canvas = createCanvas(F_M1M0S, I_W, O_M1M0Q)

#
# Traverse non-empty elements `input`
#
for m1, f_m0 in f_m1:
    o_m0 = o_m1.getPayloadRef(m1)

    for m0, f_s in f_m0:
        o_q = o_m0.getPayloadRef(m0)
        for q, o_ref in o_q:
            
            for s, f_val in f_s:
                #
                # Traverse all output locations
                #

                w = q + s
                i_val = i_w.getPayload(w)

                o_ref += i_val * f_val
                #
                # Animation bookkeeping
                #
                spacestamp = m0
                
                # TBD: Time should be counted in position not coordinates!
                timestamp = m1*W*S + q*S + s 
                
                canvas.addActivity((m1,m0,s), (w,), (m1,m0,q),
                                   spacetime=(spacestamp, timestamp))


displayTensor(O_M1M0Q)
displayCanvas(canvas)


## Testing area

For running alternative algorithms